# Import The Required Libraries

In [1]:
import json
import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load JSON File

In [2]:
JSON_PATH = "result.json"

In [4]:
json_path = Path(JSON_PATH)

raw_text = json_path.read_text(encoding="utf-8")

try:
    # Standard JSON (list or dict)
    data = json.loads(raw_text)
    if isinstance(data, dict) and "messages" in data:
        messages = data["messages"]
    else:
        messages = data
    print(f"Loaded {len(messages)} messages from JSON file.")
except json.JSONDecodeError:
    # Fallback: JSONL
    messages = []
    for i, line in enumerate(raw_text.splitlines(), start=1):
        line = line.strip()
        if not line:
            continue
        try:
            messages.append(json.loads(line))
        except json.JSONDecodeError:
            print(f"Skipping invalid JSON at line {i}")
    print(f"Loaded {len(messages)} messages from JSONL file.")

Loaded 4596 messages from JSON file.


# Flatten Telegram Text

In [6]:
rows = []

for msg in messages:
    # Combine Telegram text or text_entities into a single string
    if isinstance(msg.get("text"), list):
        full_text = "".join(
            t if isinstance(t, str) else t.get("text", "")
            for t in msg["text"]
        )
    elif isinstance(msg.get("text"), str):
        full_text = msg["text"]
    else:
        full_text = ""

    rows.append({
        "message_id": msg.get("id"),
        "date": msg.get("date"),
        "full_text": full_text
    })

records = pd.DataFrame(rows)

# Convert date to datetime (safe conversion)
records["date"] = pd.to_datetime(records["date"], errors="coerce")

records.head()

,message_id,date,full_text
0,1,2021-09-03 13:57:38,
1,12,2021-09-05 00:24:19,#خبر_مهم\n\nسلام به همه‌ی علموصی ها😍👊🏾\n\n✅این...
2,13,2021-09-05 00:24:25,
3,14,2021-09-05 00:34:39,🧑‍🏫 سید محمد شهرتاش\n🏫 #برق\n📒 بررسی ۱-بررسی۲-...
4,15,2021-09-05 00:35:51,🧑‍🏫 دکتر هاجر فلاحتی\n🏫 #مهندسی_کامپیوتر\n📒 مد...


# Extract Professor Name & Department

In [16]:
df = records[3:].copy()
df["professor_name"] = df["full_text"].str.extract(
    r"(?:🧑‍🏫\s*)?([^\n\r]+)\n?\s*🏫"
)

df["department"] = df["full_text"].str.extract(
    r"#([^\s#]+)"
)

In [17]:
df[["professor_name", "department"]].head()

,professor_name,department
3,سید محمد شهرتاش,برق
4,دکتر هاجر فلاحتی,مهندسی_کامپیوتر
5,علم الهدی,مهندسی_مواد
6,حجت قاسمی,مکانیک
7,فریبرز قاسمی,مهندسی_کامپیوتر


# Extract Numeric Ratings (0–10)

In [19]:
ratings = {
    "coherence": "پیوستگی",
    "knowledge": "دانش عمومی",
    "teaching": "انتقال",
    "management": "مدیریت",
    "responsiveness": "پاسخگویی",
    "behavior": "آداب و رفتار"
}

for col, label in ratings.items():
    df[col] = (
        df["full_text"]
        .str.extract(fr"{label}[^\d]*(\d{{1,2}})")
        .astype(float)
        .clip(0, 10)
    )

df[list(ratings.keys())].head()

,coherence,knowledge,teaching,management,responsiveness,behavior
3,8.0,9.0,7.0,9.0,8.0,10.0
4,1.0,2.0,1.0,3.0,2.0,10.0
5,9.0,8.0,10.0,9.0,10.0,10.0
6,10.0,10.0,9.0,9.0,9.0,9.0
7,8.0,10.0,8.0,10.0,7.0,9.0
